In [ ]:
# %pip install numba
# %pip install pygame

In [1]:
from numba import jit, prange
import random
import time
import pygame as p
import sys
import os
from multiprocessing import Process, Queue

pygame 2.6.1 (SDL 2.28.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


#### ChessEngine

In [2]:
class GameState:
    def __init__(self):
        self.board = [
            ["bR", "bN", "bB", "bQ", "bK", "bB", "bN", "bR"],
            ["bp", "bp", "bp", "bp", "bp", "bp", "bp", "bp"],
            ["--", "--", "--", "--", "--", "--", "--", "--"],
            ["--", "--", "--", "--", "--", "--", "--", "--"],
            ["--", "--", "--", "--", "--", "--", "--", "--"],
            ["--", "--", "--", "--", "--", "--", "--", "--"],
            ["wp", "wp", "wp", "wp", "wp", "wp", "wp", "wp"],
            ["wR", "wN", "wB", "wQ", "wK", "wB", "wN", "wR"]]
        self.moveFunctions = {"p": self.getPawnMoves, "R": self.getRookMoves, "N": self.getKnightMoves,
                              "B": self.getBishopMoves, "Q": self.getQueenMoves, "K": self.getKingMoves}
        self.white_to_move = True
        self.move_log = []
        self.white_king_location = (7, 4)
        self.black_king_location = (0, 4)
        self.checkmate = False
        self.stalemate = False
        # --- Biến cho getValidMoves ---
        self.in_check_flag = False # Đổi tên từ self.in_check để tránh nhầm lẫn với phương thức inCheck()
        self.pins = []
        self.checks = []
        # --- End biến cho getValidMoves ---
        self.enpassant_possible = ()
        self.enpassant_possible_log = [self.enpassant_possible]
        self.current_castling_rights = CastleRights(True, True, True, True)
        self.castle_rights_log = [CastleRights(self.current_castling_rights.wks, self.current_castling_rights.bks,
                                               self.current_castling_rights.wqs, self.current_castling_rights.bqs)]

    def makeMove(self, move):
        self.board[move.start_row][move.start_col] = "--"
        self.board[move.end_row][move.end_col] = move.piece_moved
        self.move_log.append(move)
        self.white_to_move = not self.white_to_move
        if move.piece_moved == "wK":
            self.white_king_location = (move.end_row, move.end_col)
        elif move.piece_moved == "bK":
            self.black_king_location = (move.end_row, move.end_col)

        if move.is_pawn_promotion:
            self.board[move.end_row][move.end_col] = move.piece_moved[0] + "Q"

        if move.is_enpassant_move:
            self.board[move.start_row][move.end_col] = "--"

        if move.piece_moved[1] == "p" and abs(move.start_row - move.end_row) == 2:
            self.enpassant_possible = ((move.start_row + move.end_row) // 2, move.start_col)
        else:
            self.enpassant_possible = ()

        if move.is_castle_move:
            if move.end_col - move.start_col == 2:
                self.board[move.end_row][move.end_col - 1] = self.board[move.end_row][move.end_col + 1]
                self.board[move.end_row][move.end_col + 1] = '--'
            else:
                self.board[move.end_row][move.end_col + 1] = self.board[move.end_row][move.end_col - 2]
                self.board[move.end_row][move.end_col - 2] = '--'

        self.enpassant_possible_log.append(self.enpassant_possible)
        self.updateCastleRights(move)
        self.castle_rights_log.append(CastleRights(self.current_castling_rights.wks, self.current_castling_rights.bks,
                                                   self.current_castling_rights.wqs, self.current_castling_rights.bqs))

    def undoMove(self):
        if len(self.move_log) != 0:
            move = self.move_log.pop()
            self.board[move.start_row][move.start_col] = move.piece_moved
            self.board[move.end_row][move.end_col] = move.piece_captured
            self.white_to_move = not self.white_to_move
            if move.piece_moved == "wK":
                self.white_king_location = (move.start_row, move.start_col)
            elif move.piece_moved == "bK":
                self.black_king_location = (move.start_row, move.start_col)
            if move.is_enpassant_move:
                self.board[move.end_row][move.end_col] = "--"
                self.board[move.start_row][move.end_col] = move.piece_captured

            self.enpassant_possible_log.pop()
            self.enpassant_possible = self.enpassant_possible_log[-1]

            self.castle_rights_log.pop()
            self.current_castling_rights = self.castle_rights_log[-1]

            if move.is_castle_move:
                if move.end_col - move.start_col == 2:
                    self.board[move.end_row][move.end_col + 1] = self.board[move.end_row][move.end_col - 1]
                    self.board[move.end_row][move.end_col - 1] = '--'
                else:
                    self.board[move.end_row][move.end_col - 2] = self.board[move.end_row][move.end_col + 1]
                    self.board[move.end_row][move.end_col + 1] = '--'
            self.checkmate = False
            self.stalemate = False

    def updateCastleRights(self, move):
        if move.piece_captured == "wR":
            if move.end_col == 0: self.current_castling_rights.wqs = False
            elif move.end_col == 7: self.current_castling_rights.wks = False
        elif move.piece_captured == "bR":
            if move.end_col == 0: self.current_castling_rights.bqs = False
            elif move.end_col == 7: self.current_castling_rights.bks = False

        if move.piece_moved == 'wK':
            self.current_castling_rights.wqs = False
            self.current_castling_rights.wks = False
        elif move.piece_moved == 'bK':
            self.current_castling_rights.bqs = False
            self.current_castling_rights.bks = False
        elif move.piece_moved == 'wR':
            if move.start_row == 7:
                if move.start_col == 0: self.current_castling_rights.wqs = False
                elif move.start_col == 7: self.current_castling_rights.wks = False
        elif move.piece_moved == 'bR':
            if move.start_row == 0:
                if move.start_col == 0: self.current_castling_rights.bqs = False
                elif move.start_col == 7: self.current_castling_rights.bks = False

    def getValidMoves(self):
        temp_castle_rights = CastleRights(self.current_castling_rights.wks, self.current_castling_rights.bks,
                                          self.current_castling_rights.wqs, self.current_castling_rights.bqs)
        
        moves = []
        # Cập nhật trạng thái self.in_check_flag, self.pins, self.checks
        self._update_pins_and_checks_status()

        king_row, king_col = self._get_current_king_location()

        if self.in_check_flag:
            if len(self.checks) == 1:
                moves = self.getAllPossibleMoves()
                check = self.checks[0]
                check_row, check_col = check[0], check[1]
                piece_checking = self.board[check_row][check_col]
                
                valid_squares = []
                if piece_checking[1] == "N":
                    valid_squares = [(check_row, check_col)]
                else:
                    for i in range(1, 8):
                        valid_square = (king_row + check[2] * i, king_col + check[3] * i)
                        valid_squares.append(valid_square)
                        if valid_square[0] == check_row and valid_square[1] == check_col:
                            break
                
                moves = self._filter_moves_when_in_check(moves, valid_squares)
            else: # Double check
                self.getKingMoves(king_row, king_col, moves)
        else: # Not in check
            moves = self.getAllPossibleMoves()
            self._add_castle_moves_if_valid(king_row, king_col, moves)

        if len(moves) == 0:
            if self.is_in_check_state(): # Sử dụng is_in_check_state thay vì inCheck trực tiếp
                self.checkmate = True
            else:
                self.stalemate = True
        else:
            self.checkmate = False
            self.stalemate = False

        self.current_castling_rights = temp_castle_rights
        return moves

    def _get_current_king_location(self):
        return self.white_king_location if self.white_to_move else self.black_king_location

    def _update_pins_and_checks_status(self):
        """Helper to call checkForPinsAndChecks and update instance variables."""
        self.in_check_flag, self.pins, self.checks = self.checkForPinsAndChecks()

    def _filter_moves_when_in_check(self, moves, valid_squares):
        """Helper to filter moves when the king is in single check."""
        filtered_moves = []
        for move in moves:
            if move.piece_moved[1] == "K": # King moves are always considered first
                filtered_moves.append(move)
            else: # Non-king moves must block the check or capture the checking piece
                if (move.end_row, move.end_col) in valid_squares:
                    filtered_moves.append(move)
        return filtered_moves
        
    def _add_castle_moves_if_valid(self, king_row, king_col, moves):
        """Helper to add castle moves if not in check and rights are available."""
        if not self.squareUnderAttack(king_row, king_col): # King is not under attack
            if self.white_to_move:
                if self.current_castling_rights.wks:
                    self.getKingsideCastleMoves(king_row, king_col, moves)
                if self.current_castling_rights.wqs:
                    self.getQueensideCastleMoves(king_row, king_col, moves)
            else: # Black to move
                if self.current_castling_rights.bks:
                    self.getKingsideCastleMoves(king_row, king_col, moves)
                if self.current_castling_rights.bqs:
                    self.getQueensideCastleMoves(king_row, king_col, moves)

    def is_in_check_state(self): # Renamed from inCheck to avoid conflict
        """Determine if the current player is in check."""
        king_row, king_col = self._get_current_king_location()
        return self.squareUnderAttack(king_row, king_col)

    def squareUnderAttack(self, row, col):
        self.white_to_move = not self.white_to_move
        opponents_moves = self.getAllPossibleMoves()
        self.white_to_move = not self.white_to_move
        for move in opponents_moves:
            if move.end_row == row and move.end_col == col:
                return True
        return False

    def getAllPossibleMoves(self):
        moves = []
        for row in range(len(self.board)):
            for col in range(len(self.board[row])):
                turn = self.board[row][col][0]
                if (turn == "w" and self.white_to_move) or \
                   (turn == "b" and not self.white_to_move):
                    piece = self.board[row][col][1]
                    # Đây là điểm có thể tối ưu hóa, các hàm con này sẽ được xem xét
                    self.moveFunctions[piece](row, col, moves)
        return moves

    def _check_line_for_pin_or_check(self, start_row, start_col, direction, ally_color, enemy_color, pins, checks):
        """
        Checks a single line of sight from the king for pins or checks.
        Returns: (is_check_found_on_line, possible_pin_on_line)
        """
        possible_pin = ()
        for i in range(1, 8):
            end_row = start_row + direction[0] * i
            end_col = start_col + direction[1] * i
            if 0 <= end_row <= 7 and 0 <= end_col <= 7:
                end_piece = self.board[end_row][end_col]
                if end_piece[0] == ally_color and end_piece[1] != "K":
                    if possible_pin == ():
                        possible_pin = (end_row, end_col, direction[0], direction[1])
                    else: # 2nd allied piece, no pin/check from this direction
                        return False, None 
                elif end_piece[0] == enemy_color:
                    enemy_type = end_piece[1]
                    # Check if the enemy piece can attack along this line
                    # j is the original index of direction in directions tuple
                    # This logic needs careful mapping if directions tuple is not passed directly
                    # For simplicity here, we assume 'j' would be known if this was a direct call
                    # from the original loop.
                    # The complex condition below needs to be adaptable or simplified.
                    # Let's assume for now the original j index mapping (0-3 ortho, 4-7 diag)
                    j = -1 # Placeholder, this indicates a dependency on original loop structure
                    # This is a simplified check, the original j mapping needs to be handled
                    if ( (direction[0] == 0 or direction[1] == 0) and enemy_type == "R") or \
                       ( (direction[0] != 0 and direction[1] != 0) and enemy_type == "B") or \
                       ( i == 1 and enemy_type == "p" and \
                         ( (enemy_color == "w" and ((direction == (-1,1) and start_row > end_row) or (direction == (-1,-1) and start_row > end_row)) ) or \
                           (enemy_color == "b" and ((direction == (1,-1) and start_row < end_row) or (direction == (1,1) and start_row < end_row))) ) ) or \
                       ( enemy_type == "Q") or \
                       ( i == 1 and enemy_type == "K"):
                        if possible_pin == (): # No piece blocking, so check
                            checks.append((end_row, end_col, direction[0], direction[1]))
                            return True, None 
                        else: # Piece blocking, so pin
                            pins.append(possible_pin)
                            return False, possible_pin
                    else: # Enemy piece not applying check along this line
                        return False, None
            else: # Off board
                return False, None
        return False, None # No check or pin found along the line

    def checkForPinsAndChecks(self):
        pins = []
        checks = []
        in_check = False
        
        ally_color, enemy_color = ('w', 'b') if self.white_to_move else ('b', 'w')
        start_row, start_col = self.white_king_location if self.white_to_move else self.black_king_location

        # Check for sliding pieces (Rook, Bishop, Queen) and pawns
        directions = ((-1, 0), (0, -1), (1, 0), (0, 1),  # Orthogonal
                      (-1, -1), (-1, 1), (1, -1), (1, 1)) # Diagonal
        
        for idx, direction in enumerate(directions):
            # We pass 'idx' to map to original 'j' for pawn checks if needed,
            # though direct pawn check logic in _check_line_for_pin_or_check needs refinement.
            # The pawn check in the original code is:
            # (i == 1 and enemy_type == "p" and ((enemy_color == "w" and 6 <= j <= 7) or (enemy_color == "b" and 4 <= j <= 5)))
            # 6,7 for white are (1,-1), (1,1) -> means pawn is at (king_row+1, king_col-1) or (king_row+1, king_col+1) attacking upwards.
            # This corresponds to white king, black pawn. Black pawn moves from row y to y+1.
            # If white king is at (r,c), black pawn at (r-1, c-1) or (r-1, c+1) attacking.
            # direction for white king to check black pawn: (-1,-1) or (-1,1)
            # These are directions[4] and directions[5]
            #
            # 4,5 for black are (-1,-1), (-1,1) -> means pawn is at (king_row-1, king_col-1) or (king_row-1, king_col+1) attacking downwards.
            # This corresponds to black king, white pawn. White pawn moves from row y to y-1.
            # If black king is at (r,c), white pawn at (r+1, c-1) or (r+1, c+1) attacking.
            # direction for black king to check white pawn: (1,-1) or (1,1)
            # These are directions[6] and directions[7]
            
            # Simplified check for _check_line_for_pin_or_check:
            # It will now rely on piece_type for rook/bishop/queen and pawn attacks are specific to i=1
            # and correct direction relative to pawn movement.
            
            # The current _check_line_for_pin_or_check has a simplified pawn check condition.
            # This part is complex and error-prone to refactor without extensive testing.
            # For now, we'll use the structure and acknowledge the pawn check part might need
            # to exactly replicate the 'j' index logic or pass 'j' itself.

            current_pins = [] # Pins found along this specific line
            current_checks = [] # Checks found along this specific line

            # The logic for identifying which piece type can attack along which direction is crucial.
            # Orthogonal directions (idx 0-3) for Rooks/Queens.
            # Diagonal directions (idx 4-7) for Bishops/Queens.
            # Pawns: specific diagonal, one step away.
            # Kings: specific any direction, one step away.
            
            # --- Iterating along a line ---
            possible_pin_on_line = ()
            for i in range(1, 8):
                end_row = start_row + direction[0] * i
                end_col = start_col + direction[1] * i
                if not (0 <= end_row <= 7 and 0 <= end_col <= 7):
                    break # Off board

                end_piece = self.board[end_row][end_col]
                if end_piece[0] == ally_color and end_piece[1] != "K":
                    if not possible_pin_on_line: # First allied piece
                        possible_pin_on_line = (end_row, end_col, direction[0], direction[1])
                    else: # Second allied piece, blocks any threat from this direction
                        break
                elif end_piece[0] == enemy_color:
                    enemy_type = end_piece[1]
                    can_attack = False
                    if (idx <= 3 and enemy_type == "R") or \
                       (idx >= 4 and enemy_type == "B") or \
                       (enemy_type == "Q") or \
                       (i == 1 and enemy_type == "K"):
                        can_attack = True
                    elif i == 1 and enemy_type == "p":
                        if enemy_color == "w": # White pawn attacking black king
                            if (direction == (1, -1) or direction == (1, 1)): # White pawn moves from row y to y-1, so king is at row+1
                                can_attack = True
                        else: # Black pawn attacking white king
                             if (direction == (-1, -1) or direction == (-1, 1)): # Black pawn moves from row y to y+1, so king is at row-1
                                can_attack = True
                    
                    if can_attack:
                        if not possible_pin_on_line: # No allied piece blocking, it's a check
                            in_check = True
                            checks.append((end_row, end_col, direction[0], direction[1]))
                        else: # Allied piece is pinned
                            pins.append(possible_pin_on_line)
                        break # Found attacking piece or pin, stop searching this line
                    else: # Enemy piece that cannot attack along this line
                        break
            # --- End iterating along a line ---

        # Check for knight checks
        knight_moves = ((-2, -1), (-2, 1), (-1, 2), (1, 2),
                        (2, -1), (2, 1), (-1, -2), (1, -2))
        for move_dir in knight_moves:
            end_row = start_row + move_dir[0]
            end_col = start_col + move_dir[1]
            if 0 <= end_row <= 7 and 0 <= end_col <= 7:
                end_piece = self.board[end_row][end_col]
                if end_piece[0] == enemy_color and end_piece[1] == "N":
                    in_check = True
                    checks.append((end_row, end_col, move_dir[0], move_dir[1]))
        
        return in_check, pins, checks

    def _get_piece_pin_info(self, row, col):
        """Helper to find if a piece at (row, col) is pinned and the pin direction."""
        for pin in self.pins:
            if pin[0] == row and pin[1] == col:
                return True, (pin[2], pin[3]) # piece_pinned, pin_direction
        return False, ()

    def _add_sliding_piece_moves(self, r, c, moves, directions, piece_type):
        """
        Generic helper for rook and bishop moves (and thus queen).
        piece_type is 'R' or 'B'. For 'Q', this will be called twice.
        """
        piece_pinned, pin_direction = self._get_piece_pin_info(r, c)
        
        # If piece is a Queen, pins are handled by both rook and bishop type calls.
        # The original code removes pin for Rook if piece is not Q, and always for Bishop.
        # This means a Queen pinned diagonally can't move orthogonally.
        # And a Queen pinned orthogonally can't move diagonally.
        # We need to ensure this behavior is preserved.

        enemy_color = "b" if self.white_to_move else "w"

        for d in directions:
            for i in range(1, 8):
                end_row, end_col = r + d[0] * i, c + d[1] * i
                if 0 <= end_row <= 7 and 0 <= end_col <= 7:
                    if piece_pinned:
                        # If pinned, can only move along the pin line (d or -d)
                        if not (d == pin_direction or d == (-pin_direction[0], -pin_direction[1])):
                            break # Cannot move in this direction due to pin
                    
                    end_piece = self.board[end_row][end_col]
                    if end_piece == "--":
                        moves.append(Move((r, c), (end_row, end_col), self.board))
                    elif end_piece[0] == enemy_color:
                        moves.append(Move((r, c), (end_row, end_col), self.board))
                        break # Cannot move further in this direction (capture)
                    else: # Friendly piece
                        break # Cannot move further
                else: # Off board
                    break
                    
    def getPawnMoves(self, row, col, moves):
        piece_pinned, pin_direction = self._get_piece_pin_info(row, col)
        # Original logic: self.pins.remove(self.pins[i]) - this modifies global state during move gen.
        # This refactoring avoids modifying self.pins here. Pin checking is done per move.

        move_amount, start_row, enemy_color = (-1, 6, "b") if self.white_to_move else (1, 1, "w")
        king_row, king_col = self.white_king_location if self.white_to_move else self.black_king_location

        # 1. Pawn Advance (1 square)
        if 0 <= row + move_amount <= 7: # Check boundary for one step
            if self.board[row + move_amount][col] == "--":
                if not piece_pinned or pin_direction == (move_amount, 0):
                    moves.append(Move((row, col), (row + move_amount, col), self.board))
                    # 2. Pawn Advance (2 squares)
                    if row == start_row and 0 <= row + 2 * move_amount <= 7: # Check boundary for two steps
                        if self.board[row + 2 * move_amount][col] == "--": # No need to check pin again for 2nd step if 1st is valid
                            moves.append(Move((row, col), (row + 2 * move_amount, col), self.board))
        
        # 3. Captures
        capture_cols = [col - 1, col + 1]
        pin_dirs_for_capture = [(move_amount, -1), (move_amount, 1)]

        for i, capture_col in enumerate(capture_cols):
            if 0 <= capture_col <= 7 and 0 <= row + move_amount <= 7: # Check boundaries for capture
                current_pin_dir_for_capture = pin_dirs_for_capture[i]
                if not piece_pinned or pin_direction == current_pin_dir_for_capture:
                    # Regular capture
                    if self.board[row + move_amount][capture_col][0] == enemy_color:
                        moves.append(Move((row, col), (row + move_amount, capture_col), self.board))
                    # En-passant capture
                    if (row + move_amount, capture_col) == self.enpassant_possible:
                        # Check for horizontal pin for en-passant (complex check)
                        # This check is to see if the capturing pawn and the pawn being captured via en-passant
                        # are on the same rank as the king, and if removing both would expose the king
                        # to a rook or queen attack horizontally.
                        can_en_passant = True # Assume true, then try to falsify
                        if king_row == row: # King is on the same rank as the capturing pawn
                            # Determine the range of squares between the king and the column of the *other* pawn
                            # (the one being captured en-passant, which is at self.enpassant_possible[1] or move.end_col)
                            # and the range of squares beyond the *other* pawn.
                            col_of_captured_ep_pawn = self.enpassant_possible[1]
                            
                            # Squares between king and the *capturing* pawn's column (col)
                            # Squares between the *capturing* pawn's col and the *captured* pawn's col (col_of_captured_ep_pawn)
                            
                            # Original logic was:
                            # if king_col < col: (king left of *capturing* pawn)
                            #   inside_range = range(king_col + 1, col -1 if capture_col < col else col) # between king and *capturing* pawn or pawn to be captured
                            #   outside_range = range(col +1 if capture_col < col else col+2 , 8) # beyond pawn to be captured or capturing pawn
                            # else: (king right of *capturing* pawn)
                            #   inside_range = range(king_col-1, col+1 if capture_col > col else col, -1)
                            #   outside_range = range(col-2 if capture_col > col else col-1, -1, -1)
                            
                            # Simpler: Check squares on king's rank from king to each side of the en passant capture
                            # If there is a horizontal attacker (R, Q) and no blockers between it and the king
                            # *after* the two pawns involved in en passant are removed.
                            
                            # Create a temporary board state to check for checks
                            temp_board = [b_row[:] for b_row in self.board]
                            temp_board[row][col] = "--" # Remove capturing pawn
                            temp_board[row][col_of_captured_ep_pawn] = "--" # Remove captured pawn (EP target square is empty)
                            
                            # Check if king is attacked horizontally after EP
                            # Check left
                            for k_c in range(king_col -1, -1, -1):
                                if temp_board[king_row][k_c] != "--":
                                    if temp_board[king_row][k_c][0] == enemy_color and \
                                       (temp_board[king_row][k_c][1] == "R" or temp_board[king_row][k_c][1] == "Q"):
                                        can_en_passant = False
                                    break # Blocker or attacker found
                            # Check right
                            if can_en_passant: # Only if not already found to be invalid
                                for k_c in range(king_col + 1, 8):
                                    if temp_board[king_row][k_c] != "--":
                                        if temp_board[king_row][k_c][0] == enemy_color and \
                                           (temp_board[king_row][k_c][1] == "R" or temp_board[king_row][k_c][1] == "Q"):
                                            can_en_passant = False
                                        break # Blocker or attacker found
                                        
                        if can_en_passant:
                             moves.append(Move((row, col), (row + move_amount, capture_col), self.board, is_enpassant_move=True))


    def getRookMoves(self, row, col, moves):
        directions = ((-1, 0), (0, -1), (1, 0), (0, 1))
        self._add_sliding_piece_moves(row, col, moves, directions, 'R')

    def getKnightMoves(self, row, col, moves):
        piece_pinned, _ = self._get_piece_pin_info(row, col) # Knight cannot move if pinned
        if piece_pinned:
            return

        knight_moves_deltas = ((-2, -1), (-2, 1), (-1, 2), (1, 2),
                               (2, -1), (2, 1), (-1, -2), (1, -2))
        ally_color = "w" if self.white_to_move else "b"
        for dr, dc in knight_moves_deltas:
            end_row, end_col = row + dr, col + dc
            if 0 <= end_row <= 7 and 0 <= end_col <= 7:
                end_piece = self.board[end_row][end_col]
                if end_piece[0] != ally_color:
                    moves.append(Move((row, col), (end_row, end_col), self.board))

    def getBishopMoves(self, row, col, moves):
        directions = ((-1, -1), (-1, 1), (1, 1), (1, -1))
        self._add_sliding_piece_moves(row, col, moves, directions, 'B')

    def getQueenMoves(self, row, col, moves):
        # Queen moves are a combination of rook and bishop moves.
        # The _add_sliding_piece_moves function needs to be aware of the piece type
        # for pin checking, or the pin checking must be perfect before calling it.
        # The original code removed pins differently for R vs B for a Q.
        # For now, we call it twice, and _get_piece_pin_info will correctly determine pin status each time.
        # The _add_sliding_piece_moves will then correctly restrict movement based on that pin.
        self.getRookMoves(row, col, moves) # Internally uses _add_sliding_piece_moves
        self.getBishopMoves(row, col, moves) # Internally uses _add_sliding_piece_moves
        
    def getKingMoves(self, row, col, moves):
        row_moves = (-1, -1, -1, 0, 0, 1, 1, 1)
        col_moves = (-1, 0, 1, -1, 1, -1, 0, 1)
        ally_color = "w" if self.white_to_move else "b"

        original_king_location = (row, col) # Store original location

        for i in range(8):
            end_row, end_col = row + row_moves[i], col + col_moves[i]
            if 0 <= end_row <= 7 and 0 <= end_col <= 7:
                end_piece = self.board[end_row][end_col]
                if end_piece[0] != ally_color:
                    # Temporarily update king's location to check for check
                    if ally_color == "w": self.white_king_location = (end_row, end_col)
                    else: self.black_king_location = (end_row, end_col)
                    
                    # Check if this move puts the king in check from opponent
                    # We need to switch turns temporarily for squareUnderAttack to work correctly from opponent's POV
                    self.white_to_move = not self.white_to_move 
                    # Also, pins/checks should be recalculated from the new king position against current player's pieces
                    # This is complex: squareUnderAttack already does getAllPossibleMoves for the opponent.
                    is_safe_square = not self.squareUnderAttack(end_row, end_col)
                    self.white_to_move = not self.white_to_move # Switch back

                    if is_safe_square:
                        moves.append(Move((row, col), (end_row, end_col), self.board))
                    
                    # Restore king's location for next iteration
                    if ally_color == "w": self.white_king_location = original_king_location
                    else: self.black_king_location = original_king_location
        # Castling moves are added separately in getValidMoves via _add_castle_moves_if_valid

    def getCastleMoves(self, row, col, moves): # Called by _add_castle_moves_if_valid
        # This function is kept for structure, but the caller `_add_castle_moves_if_valid`
        # already checks if king is under attack.
        # The rights are also checked by the caller.
        pass # Logic moved to getKingsideCastleMoves & getQueensideCastleMoves called by _add_castle_moves_if_valid

    def getKingsideCastleMoves(self, row, col, moves):
        # Assumes current_castling_rights.wks/bks is true and king is not in check.
        # Also assumes it's the correct player's turn.
        if self.board[row][col + 1] == '--' and self.board[row][col + 2] == '--':
            if not self.squareUnderAttack(row, col + 1) and \
               not self.squareUnderAttack(row, col + 2):
                moves.append(Move((row, col), (row, col + 2), self.board, is_castle_move=True))

    def getQueensideCastleMoves(self, row, col, moves):
        # Assumes current_castling_rights.wqs/bqs is true and king is not in check.
        if self.board[row][col - 1] == '--' and \
           self.board[row][col - 2] == '--' and \
           self.board[row][col - 3] == '--': # Extra square for queen side
            if not self.squareUnderAttack(row, col - 1) and \
               not self.squareUnderAttack(row, col - 2):
                moves.append(Move((row, col), (row, col - 2), self.board, is_castle_move=True))


class CastleRights:
    def __init__(self, wks, bks, wqs, bqs):
        self.wks = wks; self.bks = bks
        self.wqs = wqs; self.bqs = bqs

class Move:
    ranks_to_rows = {"1": 7, "2": 6, "3": 5, "4": 4, "5": 3, "6": 2, "7": 1, "8": 0}
    rows_to_ranks = {v: k for k, v in ranks_to_rows.items()}
    files_to_cols = {"a": 0, "b": 1, "c": 2, "d": 3, "e": 4, "f": 5, "g": 6, "h": 7}
    cols_to_files = {v: k for k, v in files_to_cols.items()}

    def __init__(self, start_square, end_square, board, is_enpassant_move=False, is_castle_move=False):
        self.start_row = start_square[0]; self.start_col = start_square[1]
        self.end_row = end_square[0]; self.end_col = end_square[1]
        self.piece_moved = board[self.start_row][self.start_col]
        self.piece_captured = board[self.end_row][self.end_col]
        
        self.is_pawn_promotion = (self.piece_moved == "wp" and self.end_row == 0) or \
                                 (self.piece_moved == "bp" and self.end_row == 7)
        
        self.is_enpassant_move = is_enpassant_move
        if self.is_enpassant_move:
            self.piece_captured = "wp" if self.piece_moved == "bp" else "bp"
        
        self.is_castle_move = is_castle_move
        self.is_capture = self.piece_captured != "--"
        self.moveID = self.start_row * 1000 + self.start_col * 100 + self.end_row * 10 + self.end_col

    def __eq__(self, other):
        return isinstance(other, Move) and self.moveID == other.moveID

    def getRankFile(self, r, c):
        return self.cols_to_files[c] + self.rows_to_ranks[r]

    def __str__(self): # Basic string representation for logging
        if self.is_castle_move:
            return "O-O" if self.end_col == 6 else "O-O-O" # Standard short castle / long castle
        
        end_square = self.getRankFile(self.end_row, self.end_col)
        
        if self.piece_moved[1] == 'p': # Pawn move
            if self.is_capture:
                return self.cols_to_files[self.start_col] + "x" + end_square
            else:
                return end_square + ("Q" if self.is_pawn_promotion else "") # Add Q for promotion
        
        # Other pieces
        move_str = self.piece_moved[1]
        if self.is_capture:
            move_str += "x"
        return move_str + end_square

#### ChessAI

In [3]:
piece_score = {"K": 0, "Q": 9, "R": 5, "B": 3, "N": 3, "p": 1}

# These score matrices can be converted to NumPy arrays later for Numba
knight_scores = [[0.0, 0.1, 0.2, 0.2, 0.2, 0.2, 0.1, 0.0],
                 [0.1, 0.3, 0.5, 0.5, 0.5, 0.5, 0.3, 0.1],
                 [0.2, 0.5, 0.6, 0.65, 0.65, 0.6, 0.5, 0.2],
                 [0.2, 0.55, 0.65, 0.7, 0.7, 0.65, 0.55, 0.2],
                 [0.2, 0.5, 0.65, 0.7, 0.7, 0.65, 0.5, 0.2],
                 [0.2, 0.55, 0.6, 0.65, 0.65, 0.6, 0.55, 0.2],
                 [0.1, 0.3, 0.5, 0.55, 0.55, 0.5, 0.3, 0.1],
                 [0.0, 0.1, 0.2, 0.2, 0.2, 0.2, 0.1, 0.0]]

bishop_scores = [[0.0, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.0],
                 [0.2, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.2],
                 [0.2, 0.4, 0.5, 0.6, 0.6, 0.5, 0.4, 0.2],
                 [0.2, 0.5, 0.5, 0.6, 0.6, 0.5, 0.5, 0.2],
                 [0.2, 0.4, 0.6, 0.6, 0.6, 0.6, 0.4, 0.2],
                 [0.2, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.2],
                 [0.2, 0.5, 0.4, 0.4, 0.4, 0.4, 0.5, 0.2],
                 [0.0, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.0]]

rook_scores = [[0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
               [0.5, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.5],
               [0.0, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.0],
               [0.0, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.0],
               [0.0, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.0],
               [0.0, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.0],
               [0.0, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.0],
               [0.25, 0.25, 0.25, 0.5, 0.5, 0.25, 0.25, 0.25]]

queen_scores = [[0.0, 0.2, 0.2, 0.3, 0.3, 0.2, 0.2, 0.0],
                [0.2, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.2],
                [0.2, 0.4, 0.5, 0.5, 0.5, 0.5, 0.4, 0.2],
                [0.3, 0.4, 0.5, 0.5, 0.5, 0.5, 0.4, 0.3],
                [0.4, 0.4, 0.5, 0.5, 0.5, 0.5, 0.4, 0.3], # Original had 0.4, I assume it's 0.3 to be symmetric
                [0.2, 0.5, 0.5, 0.5, 0.5, 0.5, 0.4, 0.2],
                [0.2, 0.4, 0.5, 0.4, 0.4, 0.4, 0.4, 0.2], # Original had 0.5, I assume it's 0.4
                [0.0, 0.2, 0.2, 0.3, 0.3, 0.2, 0.2, 0.0]]


pawn_scores = [[0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8], #Promotion row
               [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7],
               [0.3, 0.3, 0.4, 0.5, 0.5, 0.4, 0.3, 0.3],
               [0.25, 0.25, 0.3, 0.45, 0.45, 0.3, 0.25, 0.25],
               [0.2, 0.2, 0.2, 0.4, 0.4, 0.2, 0.2, 0.2],
               [0.25, 0.15, 0.1, 0.2, 0.2, 0.1, 0.15, 0.25], # Start row for black pawns after 2 steps
               [0.25, 0.3, 0.3, 0.0, 0.0, 0.3, 0.3, 0.25], # Start row for black pawns after 1 step
               [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]] # Start row (actually, this would be off-board for white pawns if they moved from row 7)
                                                        # The piece_position_scores handles flipping for black.

piece_position_scores = {"wN": knight_scores,
                         "bN": [row[::-1] for row in knight_scores[::-1]], # Flip for black
                         "wB": bishop_scores,
                         "bB": [row[::-1] for row in bishop_scores[::-1]],
                         "wQ": queen_scores,
                         "bQ": [row[::-1] for row in queen_scores[::-1]],
                         "wR": rook_scores,
                         "bR": [row[::-1] for row in rook_scores[::-1]],
                         "wp": pawn_scores,
                         "bp": [row[::-1] for row in pawn_scores[::-1]]}


CHECKMATE = 1000
STALEMATE = 0
DEPTH = 2 # Giữ DEPTH thấp để benchmark nhanh hơn

next_move_global = None # Sử dụng biến toàn cục có tiền tố để rõ ràng hơn

def findBestMove_negamax_alphabeta(game_state, valid_moves, return_queue):
    """
    Wrapper function for the NegaMax algorithm with Alpha-Beta pruning.
    Uses a global variable 'next_move_global' to store the best move found at the root.
    """
    global next_move_global
    next_move_global = None
    # random.shuffle(valid_moves) # Shuffling can be good for variety but makes benchmarking less consistent
    
    _findMoveNegaMaxAlphaBeta(game_state, valid_moves, DEPTH, -CHECKMATE, CHECKMATE,
                               1 if game_state.white_to_move else -1)
    return_queue.put(next_move_global)

def _findMoveNegaMaxAlphaBeta(game_state, valid_moves, depth, alpha, beta, turn_multiplier):
    """
    Recursive NegaMax algorithm with Alpha-Beta pruning.
    """
    global next_move_global
    if depth == 0:
        return turn_multiplier * scoreBoard(game_state) # Score from current player's perspective

    # Basic move ordering: captures first (can be improved)
    # For now, we'll keep it simple as major changes are for Numba
    # random.shuffle(valid_moves) # Moved to wrapper for consistency if used

    max_score = -CHECKMATE
    for move in valid_moves:
        game_state.makeMove(move)
        next_moves = game_state.getValidMoves() # Generate moves for the next state
        score = -_findMoveNegaMaxAlphaBeta(game_state, next_moves, depth - 1, -beta, -alpha, -turn_multiplier)
        game_state.undoMove() # Important to undo the move

        if score > max_score:
            max_score = score
            if depth == DEPTH: # If at the root of the search tree, this is a candidate for the best move
                next_move_global = move
        
        # Alpha-beta pruning
        alpha = max(alpha, max_score) # Update alpha
        if alpha >= beta:
            break # Beta cut-off
            
    return max_score

def scoreBoard(game_state):
    """
    Scores the board. Positive for white, negative for black.
    This is a good candidate for Numba.
    """
    if game_state.checkmate:
        return -CHECKMATE if game_state.white_to_move else CHECKMATE
    if game_state.stalemate:
        return STALEMATE

    current_score = 0
    for r in range(8):
        for c in range(8):
            piece = game_state.board[r][c]
            if piece != "--":
                # Material score
                piece_val = piece_score[piece[1]]
                # Positional score
                pos_score = 0
                # Ensure the piece type exists in piece_position_scores (e.g. not King)
                if piece in piece_position_scores: # piece is "wp", "bN" etc.
                     pos_score = piece_position_scores[piece][r][c]
                
                if piece[0] == 'w':
                    current_score += piece_val + pos_score
                else: # black piece
                    current_score -= (piece_val + pos_score)
    return current_score

def findRandomMove(valid_moves):
    return random.choice(valid_moves)

#### ChessMain

In [4]:
BOARD_WIDTH = BOARD_HEIGHT = 512
MOVE_LOG_PANEL_WIDTH = 250
MOVE_LOG_PANEL_HEIGHT = BOARD_HEIGHT
DIMENSION = 8
SQUARE_SIZE = BOARD_HEIGHT // DIMENSION
MAX_FPS = 15
IMAGES = {}

def loadImages():
    base_dir = os.path.dirname(os.path.abspath(sys.argv[0] if hasattr(sys, 'argv') else __file__))
    # Nếu chạy trong notebook, sys.argv[0] có thể không phải là đường dẫn file.
    # Cần điều chỉnh đường dẫn đến thư mục images nếu cần.
    # Giả sử 'images' folder is in the same directory as the notebook or script
    if not os.path.exists(os.path.join(base_dir, "images")):
         # Fallback for notebooks if images folder is in current working directory
         base_dir = os.getcwd()

    image_dir = os.path.join(base_dir, "images")
    # print(f"Attempting to load images from: {image_dir}") # Debug print

    pieces = ['wp', 'wR', 'wN', 'wB', 'wK', 'wQ', 'bp', 'bR', 'bN', 'bB', 'bK', 'bQ']
    for piece in pieces:
        image_path = os.path.join(image_dir, f"{piece}.png")
        if os.path.exists(image_path):
            try:
                image = p.image.load(image_path)
                IMAGES[piece] = p.transform.scale(image, (SQUARE_SIZE, SQUARE_SIZE))
            except Exception as e:
                print(f"Error loading image {image_path}: {e}")
        else:
            print(f"Can't find image: {image_path}")
    if not IMAGES:
        print("No images were loaded. Check the image path and ensure Pygame display is initialized.")


def main_pygame(): # Đổi tên để tránh xung đột nếu chạy trong notebook
    p.init()
    if not p.display.get_init():
        print("Pygame display module not initialized.")
        return
        
    screen = p.display.set_mode((BOARD_WIDTH + MOVE_LOG_PANEL_WIDTH, BOARD_HEIGHT))
    clock = p.time.Clock()
    screen.fill(p.Color("white"))
    
    gs = GameState()
    valid_moves = gs.getValidMoves()
    move_made = False
    animate = False 
    
    # Chỉ gọi loadImages MỘT LẦN và SAU KHI p.init()
    # Đảm bảo Pygame đã khởi tạo display mode trước khi load hình ảnh.
    if not IMAGES: # Load images if not already loaded (e.g. by benchmark cells)
        loadImages()
        if not IMAGES: # Check again
             print("Failed to load images for Pygame. Exiting.")
             p.quit()
             sys.exit()


    running = True
    square_selected = () 
    player_clicks = [] 
    game_over = False
    ai_thinking = False
    move_undone = False
    move_finder_process = None
    move_log_font = p.font.SysFont("Arial", 14, False, False) # Or any other available font
    player_one = True # Human playing white
    player_two = False # AI playing black

    while running:
        human_turn = (gs.white_to_move and player_one) or (not gs.white_to_move and player_two)
        for e in p.event.get():
            if e.type == p.QUIT:
                p.quit()
                sys.exit()
            elif e.type == p.MOUSEBUTTONDOWN:
                if not game_over and human_turn: # Allow click only if human turn
                    location = p.mouse.get_pos()
                    col = location[0] // SQUARE_SIZE
                    row = location[1] // SQUARE_SIZE
                    if square_selected == (row, col) or col >= 8:
                        square_selected = ()
                        player_clicks = []
                    else:
                        square_selected = (row, col)
                        player_clicks.append(square_selected)
                    if len(player_clicks) == 2:
                        move = Move(player_clicks[0], player_clicks[1], gs.board)
                        for i in range(len(valid_moves)):
                            if move == valid_moves[i]:
                                gs.makeMove(valid_moves[i])
                                move_made = True
                                animate = True
                                square_selected = ()
                                player_clicks = []
                                break # Found a valid move
                        if not move_made: # If the click was not a valid move
                            player_clicks = [square_selected] # Keep the last click

            elif e.type == p.KEYDOWN:
                if e.key == p.K_z: # Undo
                    if player_one and player_two: # Only allow undo if two humans
                         gs.undoMove()
                         gs.undoMove() # Undo AI's move and then player's move
                    else: # If vs AI, undo twice if it's AI's turn to make, effectively undoing AI and player.
                         gs.undoMove() # Undo last move
                         if not human_turn and len(gs.move_log) > 0 : # If it was AI's turn (so human_turn is now true), and AI moved.
                              gs.undoMove()


                    move_made = True # To regenerate valid moves
                    animate = False
                    game_over = False
                    if ai_thinking:
                        move_finder_process.terminate()
                        ai_thinking = False
                    move_undone = True # Flag to prevent AI from thinking immediately if it's its turn
                
                if e.key == p.K_r: # Reset
                    gs = GameState()
                    valid_moves = gs.getValidMoves()
                    square_selected = ()
                    player_clicks = []
                    move_made = False
                    animate = False
                    game_over = False
                    if ai_thinking:
                        move_finder_process.terminate()
                        ai_thinking = False
                    move_undone = True


        # AI Move Finder Logic
        if not game_over and not human_turn and not move_undone:
            if not ai_thinking:
                ai_thinking = True
                print("AI is thinking...")
                return_queue = Queue()
                # Sử dụng hàm AI đã refactor: findBestMove_negamax_alphabeta
                move_finder_process = Process(target=findBestMove_negamax_alphabeta, args=(gs, valid_moves, return_queue))
                move_finder_process.start()

            if not move_finder_process.is_alive() and ai_thinking: # Check ai_thinking to ensure queue is expected
                ai_move = return_queue.get()
                if ai_move is None and valid_moves: # If NegaMax fails or returns None (e.g. no moves from root)
                    print("AI couldn't find a best move, picking random.")
                    ai_move = findRandomMove(valid_moves)
                
                if ai_move: # Ensure ai_move is not None
                    gs.makeMove(ai_move)
                    print(f"AI moved: {str(ai_move)}")
                    move_made = True
                    animate = True
                else: # No valid moves for AI (stalemate/checkmate should have been caught)
                    print("AI has no valid moves.")
                    if not gs.checkmate and not gs.stalemate: # Should ideally be caught by getValidMoves
                         gs.stalemate = True # Or checkmate if king is attacked

                ai_thinking = False


        if move_made:
            if animate:
                animateMove(gs.move_log[-1], screen, gs.board, clock)
            valid_moves = gs.getValidMoves() # Regenerate valid moves
            move_made = False
            animate = False
            move_undone = False # Reset undo flag

        drawGameState(screen, gs, valid_moves, square_selected)
        
        if not game_over:
            drawMoveLog(screen, gs, move_log_font)

        if gs.checkmate:
            game_over = True
            text = "Black wins by checkmate" if gs.white_to_move else "White wins by checkmate"
            drawEndGameText(screen, text)
        elif gs.stalemate:
            game_over = True
            drawEndGameText(screen, "Stalemate")

        clock.tick(MAX_FPS)
        p.display.flip()

# --- Graphics Helper Functions (giữ nguyên) ---
def drawGameState(screen, game_state, valid_moves, square_selected):
    drawBoard(screen)
    highlightSquares(screen, game_state, valid_moves, square_selected)
    drawPieces(screen, game_state.board)

def drawBoard(screen):
    global colors
    colors = [p.Color("white"), p.Color("gray")]
    for r in range(DIMENSION):
        for c in range(DIMENSION):
            color = colors[((r + c) % 2)]
            p.draw.rect(screen, color, p.Rect(c * SQUARE_SIZE, r * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))

def highlightSquares(screen, game_state, valid_moves, square_selected):
    if (len(game_state.move_log)) > 0:
        last_move = game_state.move_log[-1]
        s = p.Surface((SQUARE_SIZE, SQUARE_SIZE))
        s.set_alpha(100)
        s.fill(p.Color('green'))
        screen.blit(s, (last_move.end_col * SQUARE_SIZE, last_move.end_row * SQUARE_SIZE))
    
    if square_selected != ():
        r, c = square_selected
        # Check if the piece belongs to the current player
        if game_state.board[r][c][0] == ('w' if game_state.white_to_move else 'b'):
            s = p.Surface((SQUARE_SIZE, SQUARE_SIZE))
            s.set_alpha(100)
            s.fill(p.Color('blue'))
            screen.blit(s, (c * SQUARE_SIZE, r * SQUARE_SIZE))
            s.fill(p.Color('yellow'))
            for move in valid_moves:
                if move.start_row == r and move.start_col == c:
                    screen.blit(s, (move.end_col * SQUARE_SIZE, move.end_row * SQUARE_SIZE))

def drawPieces(screen, board):
    for r in range(DIMENSION):
        for c in range(DIMENSION):
            piece = board[r][c]
            if piece != "--" and piece in IMAGES: # Check if image for piece exists
                screen.blit(IMAGES[piece], p.Rect(c * SQUARE_SIZE, r * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
            elif piece != "--" and piece not in IMAGES:
                 # print(f"Warning: No image for piece {piece}")
                 pass


def drawMoveLog(screen, game_state, font):
    move_log_rect = p.Rect(BOARD_WIDTH, 0, MOVE_LOG_PANEL_WIDTH, MOVE_LOG_PANEL_HEIGHT)
    p.draw.rect(screen, p.Color('black'), move_log_rect)
    move_log = game_state.move_log
    move_texts = []
    for i in range(0, len(move_log), 2):
        move_string = str(i // 2 + 1) + '. ' + str(move_log[i]) + " "
        if i + 1 < len(move_log):
            move_string += str(move_log[i + 1]) + "  "
        move_texts.append(move_string)

    moves_per_row = 2 # Điều chỉnh để vừa với panel
    padding = 5
    line_spacing = 2
    text_y = padding
    for i in range(0, len(move_texts), moves_per_row):
        text_line = ""
        for j in range(moves_per_row):
            if i + j < len(move_texts):
                text_line += move_texts[i+j]
        try:
            text_object = font.render(text_line, True, p.Color('white'))
            text_location = move_log_rect.move(padding, text_y)
            screen.blit(text_object, text_location)
            text_y += text_object.get_height() + line_spacing
        except Exception as e:
            # print(f"Error rendering move log text: {e}")
            # This can happen if font is not found or text is problematic
            pass


def drawEndGameText(screen, text):
    font = p.font.SysFont("Helvetica", 32, True, False) # Or "Arial"
    text_object = font.render(text, 0, p.Color('Gray'))
    text_location = p.Rect(0, 0, BOARD_WIDTH, BOARD_HEIGHT).move(BOARD_WIDTH / 2 - text_object.get_width() / 2,
                                                                 BOARD_HEIGHT / 2 - text_object.get_height() / 2)
    screen.blit(text_object, text_location)
    text_object = font.render(text, 0, p.Color('Black'))
    screen.blit(text_object, text_location.move(2, 2))

def animateMove(move, screen, board, clock):
    global colors # Defined in drawBoard
    d_row = move.end_row - move.start_row
    d_col = move.end_col - move.start_col
    frames_per_square = 6 # Nhanh hơn một chút
    frame_count = (abs(d_row) + abs(d_col)) * frames_per_square
    if frame_count == 0: frame_count = 1 # Ensure at least one frame for promotions etc.

    for frame in range(frame_count + 1):
        r, c = (move.start_row + d_row * frame / frame_count,
                move.start_col + d_col * frame / frame_count)
        drawBoard(screen)
        drawPieces(screen, board) # Vẽ lại bàn cờ và các quân cờ tĩnh
        
        # Erase the piece from its ending square for animation
        color = colors[(move.end_row + move.end_col) % 2]
        end_square_rect = p.Rect(move.end_col * SQUARE_SIZE, move.end_row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE)
        p.draw.rect(screen, color, end_square_rect)

        # Draw captured piece on its square if it's a capture (it will be overwritten by moving piece if it lands there)
        if move.piece_captured != '--':
            # For en passant, the captured piece is not on end_square
            if move.is_enpassant_move:
                # Captured pawn is on start_row, end_col
                en_passant_row = move.start_row
                en_passant_col = move.end_col
                # We need to draw the color of the square of the captured pawn
                captured_pawn_sq_color = colors[(en_passant_row + en_passant_col) % 2]
                p.draw.rect(screen, captured_pawn_sq_color, p.Rect(en_passant_col * SQUARE_SIZE, en_passant_row*SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
                # Then draw the captured pawn (it will be erased from board in makeMove)
                # For animation, we can show it briefly.
                # However, the board state passed to drawPieces already has the captured piece removed by makeMove logic for en passant.
                # So, we might not need to draw it here if drawPieces handles the current board state correctly.
            elif IMAGES.get(move.piece_captured): # Regular capture, draw the piece that was on end_square
                 screen.blit(IMAGES[move.piece_captured], end_square_rect)


        # Draw the moving piece
        if IMAGES.get(move.piece_moved):
            screen.blit(IMAGES[move.piece_moved], p.Rect(c * SQUARE_SIZE, r * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
        
        p.display.flip()
        clock.tick(60) # Higher tick for smoother animation

#### PlayGame

In [ ]:
#main_pygame()

KeyboardInterrupt: 

: 

#### Evaluate sequence

In [ ]:
gs_serial = GameState()
valid_moves_serial = gs_serial.getValidMoves()

print(f"Số nước đi hợp lệ ban đầu: {len(valid_moves_serial)}")

# Chuẩn bị cho việc gọi hàm AI
# findBestMove_negamax_alphabeta mong đợi một Queue, ta sẽ tạo một dummy queue
dummy_queue_serial = Queue()

# Đo thời gian thực thi findBestMove_negamax_alphabeta
start_time_serial = time.time()
findBestMove_negamax_alphabeta(gs_serial, valid_moves_serial, dummy_queue_serial)
end_time_serial = time.time()

best_move_serial = dummy_queue_serial.get() # Lấy kết quả (không bắt buộc cho benchmark thời gian)

print(f"Nước đi tốt nhất (tuần tự): {str(best_move_serial) if best_move_serial else 'None'}")
print(f"Thời gian tìm nước đi tốt nhất (tuần tự, DEPTH={DEPTH}): {end_time_serial - start_time_serial:.4f} giây")

# Thử với một vài lượt đi để có trạng thái phức tạp hơn
if best_move_serial:
    gs_serial.makeMove(best_move_serial)
    valid_moves_serial = gs_serial.getValidMoves()
    if valid_moves_serial: # If there are moves for the opponent
        opponent_move = findRandomMove(valid_moves_serial) # Opponent makes a random move
        gs_serial.makeMove(opponent_move)
        valid_moves_serial = gs_serial.getValidMoves() # New set of moves for AI
        print(f"Số nước đi hợp lệ sau vài lượt: {len(valid_moves_serial)}")
        
        if valid_moves_serial:
            dummy_queue_serial_2 = Queue()
            start_time_serial_2 = time.time()
            findBestMove_negamax_alphabeta(gs_serial, valid_moves_serial, dummy_queue_serial_2)
            end_time_serial_2 = time.time()
            best_move_serial_2 = dummy_queue_serial_2.get()
            print(f"Nước đi tốt nhất (tuần tự, sau vài lượt): {str(best_move_serial_2) if best_move_serial_2 else 'None'}")
            print(f"Thời gian tìm nước đi (tuần tự, sau vài lượt, DEPTH={DEPTH}): {end_time_serial_2 - start_time_serial_2:.4f} giây")